This is my first notebook for a Kaggle competition. In this I pretend to practice the TF skills gained during the period of training in the Google Developer Machine Learning Bootcamp.

New things learned from other noteooks:
* Feather format to rerieve data

In [3]:
import os
import gc
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [4]:
pd.options.display.max_columns = None

### Data source

In [5]:
data_path = 'data/'
feather_path = 'data/tpsoct22-feather-files/'

In [6]:
train_dtypes_file = os.path.join(data_path, 'train_dtypes.csv')
test_dtypes_file = os.path.join(data_path, 'test_dtypes.csv')
sample_submission_file = os.path.join(data_path, 'sample_submission.csv')
test_feather_file = os.path.join(feather_path, 'test.feather')

### Data analysis

In [7]:
train_dtypes_df = pd.read_csv(train_dtypes_file)
test_dtypes_df = pd.read_csv(test_dtypes_file)
cols_dtypes = {k: v for (k, v) in zip(train_dtypes_df.column, train_dtypes_df.dtype)}

In [8]:
# columns only in trainig
[c for c in train_dtypes_df['column'] if c not in test_dtypes_df['column'].values]

['game_num',
 'event_id',
 'event_time',
 'player_scoring_next',
 'team_scoring_next',
 'team_A_scoring_within_10sec',
 'team_B_scoring_within_10sec']

### Feature selection

In [9]:
targets = ['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']
useless_cols = ['game_num', 'event_id', 'event_time', 'player_scoring_next', 'team_scoring_next']
use_cols = [c for c in train_dtypes_df['column'] if c not in useless_cols]
features = [c for c in use_cols if c not in targets]

In [10]:
if False:
    train_dfs = []
    for i in range(10):
        train_file = os.path.join(data_path, f'train_{i}.csv')
        train_dfs.append(pd.read_csv(train_file, usecols=use_cols, dtype=cols_dtypes))

    pd.concat(train_dfs).reset_index(drop=True).to_feather('train.feather')
    del train_dfs
    
    test_file = os.path.join(data_path, 'test.csv')
    df = pd.read_csv(test_file, usecols=features, dtype=cols_dtypes)
    df.to_feather('test.feather')
    del df

In [11]:
#read data from multiple feather files
train_dfs = []
for i in range(10):
    train_file = os.path.join(feather_path, f'train_{i}.feather')
    train_dfs.append(pd.read_feather(train_file))
train_df = pd.concat(train_dfs).reset_index(drop=True)

# train_df.dropna(axis=0, inplace=True)
# train_df = train_df.reset_index(drop=True)

In [12]:
def get_normalized_df(df, min_data, max_data):
    # distance
    for i in range(6):
        df[f'p{i}_ball'] = np.sqrt(
           (df[f'p{i}_pos_x']-df['ball_pos_x'])**2+
           (df[f'p{i}_pos_y']-df['ball_pos_y'])**2+
           (df[f'p{i}_pos_z']-df['ball_pos_z'])**2)
    
    if min_data is None:
        min_data = df.min()
        max_data = df.max()
    
    df = (df - min_data)/(max_data - min_data)
    df.fillna(0, inplace=True)
    
    return df, min_data, max_data

In [13]:
targets_df = pd.DataFrame()
targets_df['team_A_scoring_within_10sec'] = train_df.pop('team_A_scoring_within_10sec')
targets_df['team_B_scoring_within_10sec'] = train_df.pop('team_B_scoring_within_10sec')
targets_df['no_team_scored'] = (
    targets_df['team_A_scoring_within_10sec'] == targets_df['team_B_scoring_within_10sec']
).astype('int8')

In [14]:
min_data = None
max_data = None

In [15]:
train_df, min_data, max_data = get_normalized_df(train_df, min_data, max_data)

/var/folders/7v/fh_lpbqd10dbrq8tjx6__qh40000gn/T/ipykernel_14087/1894913819.py:10: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  min_data = df.min()
/var/folders/7v/fh_lpbqd10dbrq8tjx6__qh40000gn/T/ipykernel_14087/1894913819.py:11: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  max_data = df.max()


In [16]:
train_X, val_X, train_y, val_y = train_test_split(train_df, targets_df, test_size=0.007, random_state=0)
del (train_df, targets_df)
gc.collect()

23

In [17]:
val_y

,team_A_scoring_within_10sec,team_B_scoring_within_10sec,no_team_scored
2523289,0,0,1
12016775,0,0,1
591893,0,0,1
13582302,0,0,1
12061929,0,0,1
...,...,...,...
11069204,0,0,1
5710638,0,0,1
946163,0,0,1
10696164,0,0,1


In [18]:
model = keras.models.Sequential([
    layers.BatchNormalization(input_shape = [len(train_X.columns)]),
    layers.Dense(128, activation='relu'),
    
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu'),
    
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu'),
 
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),    
    
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(3)
])

In [19]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [20]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]

In [21]:
_ = model.fit(train_X, train_y, 
              epochs=50, 
              steps_per_epoch=5000, 
              batch_size=32,
              validation_data=(val_X, val_y),
              callbacks=[callbacks])

del (train_X, val_X, train_y, val_y)
gc.collect()

Epoch 1/50


2022-10-21 18:49:05.238345: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5000/5000 [==============================] - 12s 2ms/step - loss: 0.2133 - accuracy: 0.9155 - val_loss: 0.0577 - val_accuracy: 0.9777
Epoch 2/50
5000/5000 [==============================] - 10s 2ms/step - loss: 0.0994 - accuracy: 0.9592 - val_loss: 0.0766 - val_accuracy: 0.9635
Epoch 3/50
5000/5000 [==============================] - 10s 2ms/step - loss: 0.0852 - accuracy: 0.9658 - val_loss: 0.0457 - val_accuracy: 0.9828
Epoch 4/50
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0737 - accuracy: 0.9705 - val_loss: 0.0523 - val_accuracy: 0.9804
Epoch 5/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.0675 - accuracy: 0.9729 - val_loss: 0.0574 - val_accuracy: 0.9710
Epoch 6/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.0642 - accuracy: 0.9746 - val_loss: 0.0560 - val_accuracy: 0.9783
Epoch 7/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.0599 - accuracy: 0.9757 - val_loss: 0.0466 - val_accuracy: 0.9

3576

In [22]:
test_df = pd.read_feather(test_feather_file)

In [23]:
test_df.isnull().sum().sum()

261422

In [24]:
test_df, _, _ = get_normalized_df(test_df, min_data, max_data)

In [25]:
test_df.isnull().sum().sum()

0

In [26]:
preds = model.predict(test_df)
score = tf.nn.softmax(preds)

21911/21911 [==============================] - 16s 712us/step


In [36]:
ss = pd.read_csv(sample_submission_file)
ss['team_A_scoring_within_10sec'] = score[:,0]
ss['team_B_scoring_within_10sec'] = score[:,1]
ss.to_csv('Submission.csv', index=False)
ss.head()

,id,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0,2.876272e-05,0.999971
1,1,2.327401e-07,1.000000
2,2,1.819524e-07,1.000000
3,3,7.534706e-04,0.999246
4,4,1.969745e-08,1.000000


In [35]:
ss

,id,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0,111,4
1,1,111,4
2,2,111,4
3,3,111,4
4,4,111,4
...,...,...,...
701138,701138,111,4
701139,701139,111,4
701140,701140,111,4
701141,701141,111,4


In [32]:
score[:,1]

<tf.Tensor: shape=(701143,), dtype=float32, numpy=
array([0.9999713 , 0.99999976, 0.99999976, ..., 0.99804974, 1.        ,
       0.9999999 ], dtype=float32)>